In [ ]:
!wget https://cildata.crbs.ucsd.edu/ccdb//telescience/home/CCDB_DATA_USER.portal/P2043/Experiment_6835/Subject_6837/Tissue_6840/Microscopy_6843/MP6843_img_full.zip
!wget https://cildata.crbs.ucsd.edu/ccdb//telescience/home/CCDB_DATA_USER.portal/P2043/Experiment_6835/Subject_6837/Tissue_6840/Microscopy_6843/MP6843_seg.zip
!unzip -o -d ./images MP6843_img_full.zip
!unzip -o -d ./labels MP6843_seg.zip
!rm *.zip

In [ ]:
from os.path import join
from os import listdir
import numpy as np
from PIL import Image
import torch
from torch import nn, optim, float
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import matplotlib.pyplot as plt

In [ ]:
im_root, lb_root = "./images", "./labels"
images = [join(im_root, im) for im in sorted(listdir(im_root)) if 'w1' in im]
labels = [join(lb_root, lb) for lb in sorted(listdir(lb_root)) if '_01' in lb]

In [ ]:
class CellImages(Dataset):
    def __init__(self, images, labels):
        self.images, self.labels = images, labels
        self.transform = transforms.ToTensor()

    def __len__(self):
        return len(self.images)

    def __getitem__(self, ind):
        img = self.transform(Image.open(self.images[ind]))
        lbl = self.transform(Image.open(self.labels[ind]))
        return img, lbl

In [ ]:
train = DataLoader(CellImages(images[0:71], labels[0:31]), batch_size=20)
test = DataLoader(CellImages(images[71:91], labels[31:41]), batch_size=20)

In [ ]:
for i, sample in enumerate(train):
    if i == 2:
        img, lbl = sample
        for ind in range(3):
            f, ax = plt.subplots(1,2,figsize=(10,10))
            ax[0].imshow(img[ind].numpy()[0])
            ax[1].imshow(lbl[ind].numpy()[0])
        plt.show()
        break

In [ ]:
class Block(nn.Module):
    def __init__(self, in_channel, out_channel):
        super().__init__()
        self.stack = nn.Sequential(
            nn.Conv2d(in_channel, out_channel, (3, 3)),
            nn.ReLU(),
            nn.Conv2d(out_channel, out_channel, (3, 3)),
            nn.BatchNorm2d(out_channel),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.stack(x)
        return x

In [ ]:
class UNet(nn.Module):
    def __init__(self, channels, output_size, final_output=1):
        super(UNet, self).__init__()
        self.down, self.up = channels, [c for c in reversed(channels[1:])]  # Number of feature channels
        self.down_layers, self.up_layers = nn.ModuleList(), nn.ModuleList()
        self.output_size = output_size
        self.build_layers(final_output)  # Generate list of required modules
        self.features = []

    def forward(self, x):
        output = self.expansive(self.contracting(x))
        output = F.interpolate(output, self.output_size)
        return output

    def build_layers(self, final):
        # Generate sequence layer for the expansive & contracting path
        for i in range(len(self.down) - 1):
            self.down_layers.append(Block(self.down[i], self.down[i + 1]))
            if i != max(range(len(self.down) - 1)):
                self.down_layers.append(nn.MaxPool2d((2, 2), stride=2))  # 2x2 max pooling
                self.up_layers.append(nn.ConvTranspose2d(self.up[i], self.up[i + 1], (2, 2), stride=2))
                self.up_layers.append(Block(self.up[i], self.up[i + 1]))
            else:
                self.up_layers.append(nn.Conv2d(self.up[-1], final, (1, 1)))  # 1x1 convolutional layer

    def contracting(self, model):
        """3x3 convolutions & ReLU followed by 2x2 max pooling for downsampling"""
        for i in range(0, len(self.down_layers) - 1, 2):
            block = self.down_layers[i](model)
            self.features.append(block)  # Store block outputs for concatenation
            model = self.down_layers[i + 1](block)
        model = self.down_layers[-1](model)
        return model

    def expansive(self, model):
        """Upsampling followed by 3x3 convolutions & ReLU"""
        for i in range(0, len(self.up_layers) - 1, 2):
            block = self.up_layers[i](model)
            feature = self.crop(self.features[::-1][int(i / 2)], block)
            block = torch.cat([block, feature], dim=1)  # Double the number of feature channels
            model = self.up_layers[i + 1](block)
        model = self.up_layers[-1](model)
        return model

    def crop(self, feature, block):  # Necessary due to loss of border pixels every convolution
        _, _, h, w = block.shape
        feature = transforms.CenterCrop([h, w])(feature)
        return feature

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    """Compare predictions and targets in mini-batches"""
    size = len(dataloader.dataset)
    for i, (img, lbl) in enumerate(dataloader):
        img_batch, lbl_batch = img.to('cuda'), lbl.to('cuda')
        pred = nn.Softmax(dim=1)(model(img_batch.float()))
        loss = loss_fn(pred, lbl_batch.long())

        # Backpropagation to adjust the weights and biases
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), (i + 1) * len(img_batch)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test_loop(dataloader, model, loss_fn):
    "Test the model with previously unseen, new images"
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for (img, lbl) in dataloader:
            img_batch, lbl_batch = img.to('cuda'), lbl.to('cuda')
            pred = nn.Softmax(dim=1)(model(img_batch.float()))
            test_loss += loss_fn(pred, lbl_batch.long()).item()
            correct += (pred.argmax(1) == lbl_batch).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= (size * output_size * output_size)  # Fraction of correctly classified pixels
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
output_size = (696, 520)
channels = [1, 64, 128, 256, 512, 1024]
handler = UNet(channels, output_size, 3).to('cuda')
loss_fn = nn.BCELoss()
optimiser = optim.SGD(handler.parameters(), lr=1e-3)
epochs = 5

In [ ]:
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train_loop(train, handler, loss_fn, optimiser)
    test_loop(test, handler, loss_fn)
print("Done!")
